# Inputs

In [35]:
import os
import pandas as pd
import yaml

In [36]:
DATASET_PARAMS = yaml.load(open("configs/dataset.yaml"), Loader=yaml.SafeLoader)
PREPROCESSING_PARAMS = yaml.load(open("configs/preprocessing.yaml"), Loader=yaml.SafeLoader)
MODEL_NAME = yaml.load(open("configs/model.yaml"), Loader=yaml.SafeLoader)['MODEL_NAME']
MODEL_PARAMS = yaml.load(open("configs/model.yaml"), Loader=yaml.SafeLoader)[MODEL_NAME]
TRAINING_PARAMS = yaml.load(open("configs/training.yaml"), Loader=yaml.SafeLoader)

In [37]:
categories =  DATASET_PARAMS['CATEGORIES']

BASE_DIR = os.getcwd()
DATASET_DIR = os.path.join(BASE_DIR, DATASET_PARAMS['DATA_PATH'])

TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
TEST_DIR = os.path.join(DATASET_DIR, "test")

In [38]:
from src.utils import get_device
DEVICE = get_device()
print(f"Using device: {DEVICE}")

Using device: cuda


# Dataset

In [39]:
from src.utils import generate_filenames_df

In [40]:
train_filenames_df = generate_filenames_df(TRAIN_DIR, categories)
val_filenames_df = generate_filenames_df(VAL_DIR, categories)

## Pre Processor

In [41]:
from src.preprocessing import preprocess

target_input_size = tuple(PREPROCESSING_PARAMS['INPUT_SIZE'])
train_transform = preprocess(
    target_input_size=target_input_size,
    rotation_range=PREPROCESSING_PARAMS['ROTATION_RANGE'],
    width_shift_range=PREPROCESSING_PARAMS['WIDTH_SHIFT_RANGE'],
    height_shift_range=PREPROCESSING_PARAMS['HEIGHT_SHIFT_RANGE'],
    brightness_range=PREPROCESSING_PARAMS['BRIGHTNESS_RANGE'],
    zoom_range=PREPROCESSING_PARAMS['ZOOM_RANGE'],
    horizontal_flip=PREPROCESSING_PARAMS['HORIZONTAL_FLIP'],
    vertical_flip=PREPROCESSING_PARAMS['VERTICAL_FLIP'],
    channel_shift_range=PREPROCESSING_PARAMS['CHANNEL_SHIFT_RANGE'],
    fill_mode=PREPROCESSING_PARAMS['FILL_MODE'],
    shear_range=PREPROCESSING_PARAMS['SHEAR_RANGE']
    )

val_transform = preprocess(target_input_size=target_input_size) # only rescaling

In [42]:
test_train_filenames_df =train_filenames_df.reset_index(drop=True).sample(frac=1, random_state=42).reset_index(drop=True)
train_filenames_df

,image_path,class
0,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,1
1,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,1
2,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,1
3,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,1
4,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,1
...,...,...
1155,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1156,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1157,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1158,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3


In [43]:
test_train_filenames_df.iloc[1000:]

,image_path,class
1000,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,2
1001,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1002,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,2
1003,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,2
1004,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
...,...,...
1155,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1156,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1157,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3
1158,c:\Users\Yuno\Documents\Kuliah Sarjana\Project...,3


In [44]:
len(test_train_filenames_df[test_train_filenames_df['class'] == '1'])

254

## Train loader

In [45]:
from src.loader import Loader
train_loader = Loader(train_filenames_df, 
                     batch_size=TRAINING_PARAMS['BATCH_SIZE'], 
                     num_workers=TRAINING_PARAMS['NUM_WORKERS'], 
                     transform=train_transform, 
                     shuffle=TRAINING_PARAMS['SHUFFLE'])

val_loader = Loader(val_filenames_df, 
                    batch_size=TRAINING_PARAMS['BATCH_SIZE'], 
                    num_workers=TRAINING_PARAMS['NUM_WORKERS'], 
                    transform=val_transform)


In [46]:
# # Get a sample batch from train_loader
# import matplotlib.pyplot as plt
# import torch

# # Get first batch
# images, labels = next(iter(train_loader))

# # Select first image and label from batch
# sample_image = images[0]
# sample_label = labels[0]

# # Convert tensor to numpy for displaying
# # Move channel dimension from [C,H,W] to [H,W,C] for plotting
# sample_image_np = sample_image.permute(1,2,0).numpy()

# # Multiply by 255 to restore original pixel values
# sample_image_np = sample_image_np * 255

# # Plot
# plt.figure(figsize=(8,8))
# plt.imshow(sample_image_np)
# plt.title(f'Label: {sample_label}')
# plt.axis('off')
# plt.show()


# Model

In [47]:
num_classes = len(categories)

### EfficientCapsNet

In [48]:
if MODEL_NAME == "CAPSNET":
    from src.model import EfficientCapsNet
    from src.loss import MarginLoss, marginLoss

    model = EfficientCapsNet(input_size=(MODEL_PARAMS['INPUT_SIZE']))
    loss = MarginLoss()
    # loss = marginLoss

### DenseNet

In [49]:
if MODEL_NAME == "DENSENET121":
    from src.densenet import DenseNet121
    from torch.nn import CrossEntropyLoss

    model = DenseNet121(num_classes=num_classes, dropout_rate=MODEL_PARAMS['DROPOUT_RATE'])
    loss = CrossEntropyLoss()   

# Training

## Metrics

In [50]:
# Optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=TRAINING_PARAMS['LEARNING_RATE'])

# use torcheval metrics
# metrics
from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassAUROC,
    MulticlassAUPRC,
    MulticlassRecall
)

# Metrics
from src.metrics import (
    MulticlassMCC,
    MulticlassSpecificity
)

metrics = {
    "mcc": MulticlassMCC(num_classes=num_classes, device=DEVICE),
    "auprc": MulticlassAUPRC(num_classes=num_classes, average= TRAINING_PARAMS['AVERAGE'], device=DEVICE),
    "auroc": MulticlassAUROC(num_classes=num_classes, average= TRAINING_PARAMS['AVERAGE'], device=DEVICE),
    "accuracy": MulticlassAccuracy(num_classes=num_classes, average= TRAINING_PARAMS['AVERAGE'], device=DEVICE),
    "f1_score": MulticlassF1Score(num_classes=num_classes, average= TRAINING_PARAMS['AVERAGE'], device=DEVICE),
    "precision": MulticlassPrecision(num_classes=num_classes, average=TRAINING_PARAMS['AVERAGE'], device = DEVICE),
    "recall": MulticlassRecall(num_classes=num_classes, average=TRAINING_PARAMS['AVERAGE'], device = DEVICE),
    "specificity": MulticlassSpecificity(num_classes=num_classes, average=TRAINING_PARAMS['AVERAGE'], device = DEVICE)
}

In [51]:
from src.train import train

history = train(model=model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    criterion=loss, 
    optimizer=optimizer, 
    num_epochs=TRAINING_PARAMS['NUM_EPOCHS'], 
    device=DEVICE,
    metrics=metrics,
    print_every=TRAINING_PARAMS['PRINT_EVERY'],
    save_patience=TRAINING_PARAMS['SAVE_PATIENCE'],
    save_path=TRAINING_PARAMS['SAVE_PATH'],
    save_model=TRAINING_PARAMS['SAVE_MODEL'],
    save_metrics=TRAINING_PARAMS['SAVE_METRICS']
    )

Training...

Epoch 1/10
Epoch 1/10, Batch 1/1160, Train Loss: 1.1663
Epoch 1/10, Batch 2/1160, Train Loss: 1.5066
Epoch 1/10, Batch 3/1160, Train Loss: 1.6037
Epoch 1/10, Batch 4/1160, Train Loss: 1.6047
Epoch 1/10, Batch 5/1160, Train Loss: 1.5992
Epoch 1/10, Batch 6/1160, Train Loss: 1.5718
Epoch 1/10, Batch 7/1160, Train Loss: 1.5280
Epoch 1/10, Batch 8/1160, Train Loss: 1.4526
Epoch 1/10, Batch 9/1160, Train Loss: 1.4254
Epoch 1/10, Batch 10/1160, Train Loss: 1.4178
Epoch 1/10, Batch 11/1160, Train Loss: 1.3792
Epoch 1/10, Batch 12/1160, Train Loss: 1.3772
Epoch 1/10, Batch 13/1160, Train Loss: 1.3547
Epoch 1/10, Batch 14/1160, Train Loss: 1.3352
Epoch 1/10, Batch 15/1160, Train Loss: 1.3085
Epoch 1/10, Batch 16/1160, Train Loss: 1.2927
Epoch 1/10, Batch 17/1160, Train Loss: 1.2835
Epoch 1/10, Batch 18/1160, Train Loss: 1.2774
Epoch 1/10, Batch 19/1160, Train Loss: 1.2724
Epoch 1/10, Batch 20/1160, Train Loss: 1.2767
Epoch 1/10, Batch 21/1160, Train Loss: 1.2810
Epoch 1/10, Batch 2

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
